# BT4Py TT5 Top-1

In [1]:
%load_ext autoreload
%autoreload 2

%pwd
%cd /home/benji/Documents/Uni/heidelberg/05/masterarbeit/impls/scripts/experiments

/home/benji/Documents/Uni/heidelberg/05/masterarbeit/impls/scripts/experiments


In [2]:
import polars as pl

pl.Config.set_fmt_str_lengths(300)
pl.Config.set_tbl_rows(n=50)

polars.config.Config

In [3]:
import pathlib

from scripts.common.schemas import TypeCollectionCategory
from scripts.infer.structure import DatasetFolderStructure

tool = "typilusN1"
dataset = DatasetFolderStructure(pathlib.Path(
    "/home/benji/Documents/Uni/heidelberg/05/masterarbeit/datasets/better-types-4-py-dataset"
))


2023-06-24 13:55:08.690193: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-24 13:55:09.735985: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import logging
from importlib import reload

logging.shutdown()
reload(logging)

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
for handler in logger.handlers:
    logger.removeHandler(handler)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

ch.setFormatter(logging.Formatter(f"[{tool} @ %(levelname)s]: %(message)s"))
logger.addHandler(ch)

logger.info("Hello World!")


[typilusN1 @ INFO]: Hello World!


In [5]:
# TypeT5 paper has Typilus evaluated ONLY on common types, as the freely trained model has no ability of predicting 
#
# The original Typilus model can predict from a set of common types as well as (nonparametric) userdefined types, 
# but their released model can only predict common types, so we only evaluate its performance on common types. - TypeT5 Paper
import libcst

from typet5.model import ModelWrapper
model = ModelWrapper.load_from_hub("MrVPlusOne/TypeT5-v7")
common_types = model.common_type_names

logger.debug(f"{len(common_types)} common types are known by TypeT5")
logger.debug(f"{common_types}")



Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

[typilusN1 @ DEBUG]: 100 common types are known by TypeT5
[typilusN1 @ DEBUG]: {'Vertex', 'ndarray', 'Any', 'Path', 'Settings', 'DataT', 'PartyID', 'Generator', 'Issue', 'Tensor', 'Iterable', 'Article', 'int', 'Result', 'Parameter', 'Exception', 'Client', 'Set', 'Collection', 'timedelta', 'float', 'None', 'Model', 'MagicMock', 'Configuration', 'Address', 'URL', 'Decimal', 'Node', 'object', '_MakeClient', 'Session', 'Logger', 'Qubit', 'Nvim', 'Sequence', 'bool', 'Request', 'Expr', 'Namespace', 'Response', '...', 'Variable', 'ArgumentParser', 'Union', 'Awaitable', 'User', 'str', 'Message', 'Outcome', 'Iterator', 'Flask', 'Callable', 'IO', 'datetime', 'Container', 'Candidates', 'CWLObjectType', 'GlobalState', 'ID', 'Root', 'HttpRequest', 'Literal', 'Context', 'Field', 'Type', 'Scope', 'Source', 'Redis', 'UserID', 'State', 'Tuple', 'BytesIO', 'T', '<List>', 'date', 'AsyncIterator', 'Connection', 'DataFrame', 'Application', 'Config', 'BaseException', 'Mock', 'Task', 'Table', 'Mapping', 'Tok

#  Prediction Metrics for Full Accuracy

In [85]:
import pathlib, shutil

from typet5.experiments.utils import remove_newer_syntax_for_repo
from typet5.experiments.typilus import TypilusSupportedSyntax

repos_dir = dataset.dataset_root
typilus_rewritten_repos = pathlib.Path() / "typilus-repos"
if not typilus_rewritten_repos.is_dir():
    logger.info("Creating rewritten repos")
    shutil.copytree(repos_dir / "repos" / "test", typilus_rewritten_repos)
    remove_newer_syntax_for_repo(typilus_rewritten_repos, TypilusSupportedSyntax)

logger.info(f"Rewritten repos are at {typilus_rewritten_repos.resolve()}")

[typilusN1 @ INFO]: Creating rewritten repos
INFO:__main__:Creating rewritten repos
Removing newer syntax: 100%|█████████████████████████████████████████████████████| 1636/1636 [00:14<00:00, 116.02it/s]
[typilusN1 @ INFO]: Rewritten repos are at /home/benji/Documents/Uni/heidelberg/05/masterarbeit/impls/scripts/experiments/typilus-repos
INFO:__main__:Rewritten repos are at /home/benji/Documents/Uni/heidelberg/05/masterarbeit/impls/scripts/experiments/typilus-repos


1636 / 1636 files have been rewritten.


In [90]:
from importlib import reload

from libcst import helpers as h
import tqdm

from typet5.experiments.typilus import eval_typilus_on_repos, TypilusSupportedSyntax

from typet5.experiments import typilus
reload(typilus)
from typet5.static_analysis import PythonProject, SignatureMap, AccuracyMetric, SignatureErrorAnalysis
from typet5.experiments.typet5 import accs_as_table_row
from typet5.visualization import pretty_print_dict

import itertools, shutil
import io, gzip, json, pprint

test_repos = [p for p in typilus_rewritten_repos.iterdir() if p.is_dir()]

def typet5_metrics_4_typilus(task: TypeCollectionCategory) -> None:
    def transform_typilus_pred(prediction: dict) -> dict:
        # print(prediction)
        ty, prob = prediction["predicted_annotation_logprob_dist"][0]
        prediction["pred"] = ty
        
        del prediction["predicted_annotation_logprob_dist"]
        return prediction
        
        
    from scripts.common.output import InferenceArtifactIO
    test_set = dataset.test_set()

    proj2datasets = [
        (
            project,
            InferenceArtifactIO(
                artifact_root=pathlib.Path(),
                dataset=dataset,
                repository=project,
                tool_name=tool,
                task=task,
            ),
        )
        for project in test_set
    ]
    existing = dict(
        (project, artifact)
        for project, artifact in proj2datasets
        if artifact.full_location().exists()
    )

    assignments = []
    projects = []

    for (project, artifact) in tqdm.tqdm(existing.items(), desc=f"Loading predictions from {task}"):
        typilus_predictions, = artifact.read()
        
        with gzip.open(io.BytesIO(typilus_predictions)) as f:
            decoded = json.loads(f.read().decode("utf-8"))
        typilus_json = list(map(transform_typilus_pred, decoded))
        
        ordered_by_file = sorted(typilus_json, key=lambda p: p["provenance"])
        transformed = {
            f"{key}": list(predictions) 
            for key, predictions in itertools.groupby(ordered_by_file, key=lambda p: p["provenance"])
        }
        # pprint.pprint(transformed)

        assignments.append(transformed)

    
    accs = typilus.analyze_typilus_predictions(
        typilus_outputs=assignments,
        repo_roots=test_repos,
        metrics=AccuracyMetric.default_metrics(common_types),
    )
    accs_as_table_row(accs)
    pretty_print_dict(accs)
    

In [92]:
typet5_metrics_4_typilus(task=TypeCollectionCategory.CALLABLE_PARAMETER)

Loading predictions from CALLABLE_PARAMETER: 100%|███████████████████████████████████| 50/50 [00:00<00:00, 889.71it/s]


Accuracies on all types:
header:  ['full.all', 'calibrated.all', 'calibrated.simple', 'calibrated.complex', 'base.all']
14.42 & 18.30 & 21.66 & 0.00 & 27.08
Accuracies on common types:
header:  ['full.all', 'calibrated.all', 'calibrated.simple', 'calibrated.complex', 'base.all']
14.42 & 18.30 & 21.66 & 0.00 & 27.08
Accuracies on rare types:
header:  ['full.all', 'calibrated.all', 'calibrated.simple', 'calibrated.complex', 'base.all']
nan & nan & N/A & N/A & nan
full_acc:
   full_acc: 14.42% (count=1.5k)
   full_acc_by_cat:
      FuncArg: 12.65% (count=917)
      FuncReturn: 17.80% (count=500)
      ClassAtribute: 17.14% (count=70)
      GlobalVar: 6.25% (count=32)
   full_acc_by_simple:
      complex: 3.05% (count=295)
      simple: 17.16% (count=1.2k)
   full_acc_label_size: 1.6103
   full_acc_pred_size: 1.3752
   full_acc_ignored_labels: 0
   n_missing: 14267
   n_skipped_rare: 709
full_acc_common:
   full_acc_common: 14.42% (count=1.5k)
   full_acc_common_by_cat:
      FuncArg: 12.6

In [99]:
reload(typilus)
typet5_metrics_4_typilus(task=TypeCollectionCategory.CALLABLE_RETURN)

Loading predictions from CALLABLE_RETURN: 100%|█████████████████████████████████████| 50/50 [00:00<00:00, 1090.96it/s]


Accuracies on all types:
header:  ['full.all', 'calibrated.all', 'calibrated.simple', 'calibrated.complex', 'base.all']
12.29 & 19.54 & 22.97 & 0.00 & 27.53
Accuracies on common types:
header:  ['full.all', 'calibrated.all', 'calibrated.simple', 'calibrated.complex', 'base.all']
12.29 & 19.54 & 22.97 & 0.00 & 27.53
Accuracies on rare types:
header:  ['full.all', 'calibrated.all', 'calibrated.simple', 'calibrated.complex', 'base.all']
nan & nan & N/A & N/A & nan
full_acc:
   full_acc: 12.29% (count=2.4k)
   full_acc_by_cat:
      FuncArg: 15.88% (count=1.2k)
      FuncReturn: 8.23% (count=1.1k)
      ClassAtribute: 17.14% (count=70)
      GlobalVar: 6.25% (count=32)
   full_acc_by_simple:
      complex: 1.84% (count=434)
      simple: 14.64% (count=1.9k)
   full_acc_label_size: 1.4432
   full_acc_pred_size: 1.3409
   full_acc_ignored_labels: 0
   n_missing: 241
   n_skipped_rare: 784
full_acc_common:
   full_acc_common: 12.29% (count=2.4k)
   full_acc_common_by_cat:
      FuncArg: 15.88

In [93]:
typet5_metrics_4_typilus(task=TypeCollectionCategory.VARIABLE)

Loading predictions from VARIABLE: 100%|█████████████████████████████████████████████| 50/50 [00:00<00:00, 872.33it/s]


Accuracies on all types:
header:  ['full.all', 'calibrated.all', 'calibrated.simple', 'calibrated.complex', 'base.all']
16.50 & 19.71 & 23.25 & 0.00 & 27.83
Accuracies on common types:
header:  ['full.all', 'calibrated.all', 'calibrated.simple', 'calibrated.complex', 'base.all']
16.50 & 19.71 & 23.25 & 0.00 & 27.83
Accuracies on rare types:
header:  ['full.all', 'calibrated.all', 'calibrated.simple', 'calibrated.complex', 'base.all']
nan & nan & N/A & N/A & nan
full_acc:
   full_acc: 16.50% (count=1.8k)
   full_acc_by_cat:
      FuncArg: 16.09% (count=1.2k)
      FuncReturn: 18.07% (count=498)
      ClassAtribute: 16.90% (count=71)
      GlobalVar: 6.25% (count=32)
   full_acc_by_simple:
      complex: 2.59% (count=347)
      simple: 19.85% (count=1.4k)
   full_acc_label_size: 1.5889
   full_acc_pred_size: 1.3635
   full_acc_ignored_labels: 0
   n_missing: 13936
   n_skipped_rare: 771
full_acc_common:
   full_acc_common: 16.50% (count=1.8k)
   full_acc_common_by_cat:
      FuncArg: 16.